# Imports

In [1]:
import pandas as pd # to interact with dataframes 
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder #for preprocessing
import joblib #to save encoders and models 
import os #to interact with hardware and create folders 
import warnings
warnings.simplefilter(action='ignore')

# Constants

In [2]:
# Initialize filepaths 
LOCALITY_ENCODER_PATH = 'Objects/Encoders/LabelEncoder/'
LAYOUT_ENCODER_PATH = 'Objects/Encoders/OrdinalEncoder/layout_type/'
PROPERTY_ENCODER_PATH = 'Objects/Encoders/OrdinalEncoder/property_type/'
SELLER_ENCODER_PATH = 'Objects/Encoders/OrdinalEncoder/seller_type/'
FURNITURE_ENCODER_PATH = 'Objects/Encoders/OrdinalEncoder/furniture_encoders/'
CLEAN_SQL_PATH = 'sql_files/clean/'
PREPROCESSED_FILE_PATH = 'preprocessed/'

# Reading the Data

In [3]:
data = pd.read_csv('_All_Cities_Cleaned.csv') # read the data 

In [4]:
data.head() # see first 5 rows of the data 

,seller_type,bedroom,layout_type,property_type,locality,price,area,furnish_type,bathroom,city
0,OWNER,2.0,BHK,Apartment,Bodakdev,20000.0,1450.0,Furnished,2.0,Ahmedabad
1,OWNER,1.0,RK,Studio Apartment,CG Road,7350.0,210.0,Semi-Furnished,1.0,Ahmedabad
2,OWNER,3.0,BHK,Apartment,Jodhpur,22000.0,1900.0,Unfurnished,3.0,Ahmedabad
3,OWNER,2.0,BHK,Independent House,Sanand,13000.0,1285.0,Semi-Furnished,2.0,Ahmedabad
4,OWNER,2.0,BHK,Independent House,Navrangpura,18000.0,1600.0,Furnished,2.0,Ahmedabad


In [5]:
# create a copy of the subsets of the dataframes for each city
ahmedabad = data[data['city'] == 'Ahmedabad'].copy()
bangalore = data[data['city'] == 'Bangalore'].copy()
chennai = data[data['city'] == 'Chennai'].copy()
delhi = data[data['city'] == 'Delhi'].copy()
hyderabad = data[data['city'] == 'Hyderabad'].copy()
kolkata = data[data['city'] == 'Kolkata'].copy()
mumbai = data[data['city'] == 'Mumbai'].copy()
pune = data[data['city'] == 'Pune'].copy()

In [6]:
df_list = [ahmedabad,bangalore,chennai,delhi,hyderabad,kolkata,mumbai,pune] #store the dataframes in a list
cities = ['AHEMDABAD','BANGALORE','CHENNAI','DELHI','HYDERABAD','KOLKATA','MUMBAI','PUNE'] # store the city names in a list
df_dict = dict(zip(cities,df_list)) #create a dictionary of dataframes with cities as key

In [7]:
# for storing encoders while preprocessing 
locality_encoder_dict = {}
furnish_type_encoder_dict = {}
seller_type_encoder_dict = {}
layout_type_encoder_dict = {}
property_type_encoder_dict = {}
def preprocess(df_dict):
    """
    Cleans and preprocesses the data for future use 
    Cleaning:
        - Removes duplicate rows (if they exist)
        - Removes the city column as the dataframes are already separated 
        - Removes outliers on the basis of price column using lower limit as (Q1-1.5*IQR) and 
        upper limit as (Q3+1.5*IQR)
    Preprocessing:
        - Uses label encoding for locality column as order does not matter in locality
        - Uses ordianal encoding for furnish_type, layout_type, seller_type and property_type as the 
        order matters in these columns for predicting the rent prices
    Args:
        df_dict - the dictionary containing cities as keys and the dataframe corresponding to them as values 
    Returns:
        None 
    """
    for city, df in df_dict.items():
        # Cleaning the data 
        # Dropping the city column
        df.drop(['city'], axis=1, inplace=True)
        # Dropping duplicate rows (if they exist)
        df.drop_duplicates(inplace=True)
        # Renaming the columns 
        cols = ['SELLER TYPE','BEDROOM','LAYOUT TYPE','PROPERTY TYPE','LOCALITY','PRICE','AREA','FURNISH TYPE','BATHROOM']
        df.columns = cols
        # Removing outliers 
        desc = df['PRICE'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['PRICE']>=lower_lim)&(df['PRICE']<=upper_lim)]
        # Renaming the columns 
        # since the data is cleaned now, we can store it in clean_dict 
        clean_df = df.copy()

        # write the data to SQL file to insert into the CLEAN database
        clean_sql_filepath = os.path.join(CLEAN_SQL_PATH, f'{city}.sql')

        with open(clean_sql_filepath, 'w+') as f:    
            for idx, row in clean_df.iterrows():
                query = f"""
    INSERT INTO {city} (SELLER_TYPE, BEDROOM, LAYOUT, PROPERTY_TYPE, LOCALITY, PRICE, AREA, FURNISH_TYPE, BATHROOM)
    VALUES 
    {tuple(row.values)};\n
                """
                f.write(query)
        # Preprocessing the data 
        locality_encoder = LabelEncoder()
        df['LOCALITY'] = locality_encoder.fit_transform(df['LOCALITY'])
        locality_encoder_dict[city] = locality_encoder
        if not os.path.exists(LOCALITY_ENCODER_PATH):
            joblib.dump(locality_encoder)
        
        ordinal_encoder_cols = ['SELLER TYPE','LAYOUT TYPE','PROPERTY TYPE','FURNISH TYPE']        
        ord_enc_dict = {
            'SELLER TYPE':seller_type_encoder_dict,
            'LAYOUT TYPE':layout_type_encoder_dict,
            'PROPERTY TYPE':property_type_encoder_dict,
            'FURNISH TYPE':furnish_type_encoder_dict
        }
        for col in ordinal_encoder_cols:
            cat = [df.groupby(by=[col])['PRICE'].mean().sort_values(ascending=True).index]
            col_encoder = OrdinalEncoder(categories=cat)
            df[col] = col_encoder.fit_transform(df[[col]])
            ord_enc_dict[col][city] = col_encoder
        paths = {
            'SELLER TYPE': os.path.join(SELLER_ENCODER_PATH, f'{city}_seller_type_encoder.pkl'),
            'LAYOUT TYPE': os.path.join(LAYOUT_ENCODER_PATH,f'{city}_layout_type_encoder.pkl'),
            'PROPERTY TYPE': os.path.join(PROPERTY_ENCODER_PATH,f'{city}_property_type_encoder.pkl'),
            'FURNISH TYPE': os.path.join(FURNITURE_ENCODER_PATH, f'{city}_furnish_type_encoder.pkl')
        }

        if not os.path.exists(FURNITURE_ENCODER_PATH): #check if the desired file path exists
            os.makedirs(FURNITURE_ENCODER_PATH) #if not then make one 

        if not os.path.exists(SELLER_ENCODER_PATH):
            os.makedirs(SELLER_ENCODER_PATH)
        
        if not os.path.exists(LAYOUT_ENCODER_PATH):
            os.makedirs(LAYOUT_ENCODER_PATH)
        
        if not os.path.exists(PROPERTY_ENCODER_PATH):
            os.makedirs(PROPERTY_ENCODER_PATH)
     
        for col in ordinal_encoder_cols:
            joblib.dump(ord_enc_dict[col][city], paths[col])

        # Saving the dataframes 
        if not os.path.exists(PREPROCESSED_FILE_PATH):
            os.makedirs(PREPROCESSED_FILE_PATH)
        df.to_csv(os.path.join(PREPROCESSED_FILE_PATH, f'{city}.csv'),index=False)

In [8]:
preprocess(df_dict) # call the preprocess function 